# Whisper

De nombreux tutoriaux qui fleurissent :

- https://www.css.cnrs.fr/whisper-pour-retranscrire-des-entretiens/
- https://analyzingalpha.com/openai-whisper-python-tutorial

Intérêt ? 

- utiliser un modèle
- le faire en local

## Un coup d'oeil sur ce qu'est whisper

https://github.com/openai/whisper

## Installer la bibliothèque Python

décommenter la ligne pour installer

In [2]:
#pip install openai-whisper

## Charger un modèle

In [3]:
import whisper
model1 = whisper.load_model("tiny")
#model2 = whisper.load_model("medium")

100%|█████████████████████████████████████| 72.1M/72.1M [00:24<00:00, 3.05MiB/s]


C'est un objet de type whisper

In [5]:
type(model1)

whisper.model.Whisper

In [6]:
model1

Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-3): 4 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=384, out_features=384, bias=True)
          (key): Linear(in_features=384, out_features=384, bias=False)
          (value): Linear(in_features=384, out_features=384, bias=True)
          (out): Linear(in_features=384, out_features=384, bias=True)
        )
        (attn_ln): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=1536, out_features=384, bias=True)
        )
        (mlp_ln): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm((384,), eps=1e-05,

## Modifier ses mp3

Créer un extrait de 30 s

In [8]:
#pip install pydub

In [11]:
#from pydub import AudioSegment
import pydub

In [12]:
itw = pydub.AudioSegment.from_mp3("audio_cv_i.MP3")

Extraire entre la seconde 30 et la seconde 90

In [12]:
extract = itw[30*1000:90*1000]

Ecrire sur le disque dur

In [13]:
AudioSegment.export(extract,"sample.mp3")

<_io.BufferedRandom name='sample.mp3'>

## Retranscrire l'extrait

In [10]:
# Si jamais il y a une dépendance qui manque
#!pip install ffmpeg-python

Utiliser la fonction de transcription

In [13]:
result = model1.transcribe("sample.mp3")

/home/emilien/anaconda3/lib/python3.9/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [14]:
result["text"]

" On les parant qui était là, elle n'est pas qu'on prie cette réalisation. Et très vite, ce sont orientés. Une fois que la maison était en cours, en fait, sur tous les problèmes de qualité de vie. Dans lesquels il y avait le social, la formation, la vie à l'hôpital. La prise en charge, etc. Pas toujours, donc je vous demande de répondre de sa part. Pas toujours avec la corps, d'ailleurs, des médecins. C'est-à-dire que on n'était pas en conflit, mais non non, c'est pas ça du tout. On n'était pas toujours. On connaît ça des plus... C'était en 1990. Et à ce moment-là, la vie d'atrice à Gustavouille-Cier, elle avait une place importante. Oui, oui, déjà. Avec elle. Voilà. Donc voilà. Et à cette même période, même quelques fois, quelques années avant, toutes les associations de parant qui vont se réunir ensuite dans les aspects, qui ont de mine de près, de quoi? Ce sont créatures de ces mêmes projets. C'est-à-dire d'abord premièrement..."

Essai d'un autre modèle plus précis

In [17]:
result = model2.transcribe("sample.mp3")
result["text"]

" Les parents qui étaient là à l'époque ont pris cette réalisation et très vite se sont orientés, une fois que la maison était en cours, sur tous les problèmes de qualité de vie dans lesquels il y avait le social, la formation, la vie à l'hôpital, donc la prise en charge, etc. Pas toujours, donc je vous demande de répondre ça partout, pas toujours avec l'accord des médecins. C'est-à-dire que c'était pas en conflit, non c'est pas ça du tout, mais on n'était pas toujours. C'était en 89, et à ce moment-là la pédiatrie à Augustin-Roussier avait une place importante ? Oui, oui, déjà. Elle avait déjà une place. Voilà, donc voilà, et dans cette même période, mais même quelques fois quelques années avant, toutes les associations de parents qui vont se réunir ensuite dans l'UNAPEC, mais en 2003, se sont créées autour de ces mêmes projets, c'est-à-dire d'abord, premièrement,"

Mise sous la forme d'un tableau

In [16]:
import pandas as pd
speech = pd.DataFrame.from_dict(result['segments'])
speech.head()

,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob
0,0,0,0.00,3.98,"On les parant qui était là, elle n'est pas qu...","[50364, 1282, 1512, 971, 394, 1956, 11806, 368...",0.0,-0.518136,1.580756,0.255956
1,1,0,3.98,5.58,"Et très vite, ce sont orientés.","[50563, 3790, 5732, 24462, 11, 1769, 4900, 857...",0.0,-0.518136,1.580756,0.255956
2,2,0,5.58,7.48,"Une fois que la maison était en cours, en fait,","[50643, 16701, 9576, 631, 635, 28511, 11806, 4...",0.0,-0.518136,1.580756,0.255956
3,3,0,7.48,9.88,sur tous les problèmes de qualité de vie.,"[50738, 1022, 8317, 1512, 37317, 368, 42106, 3...",0.0,-0.518136,1.580756,0.255956
4,4,0,9.88,13.52,"Dans lesquels il y avait le social,","[50858, 16897, 1512, 358, 1625, 1930, 288, 118...",0.0,-0.518136,1.580756,0.255956


## Mise en forme et sauvegarde

(inspiration Yacine Chitour) 

In [17]:
def convertir(seconds):
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = int(seconds % 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

In [20]:
convertir(100000)

'27:46:40'

In [25]:
# Chemin où vous souhaitez enregistrer votre transcription d'entretien en format .txt
entretien_transcrit = "entretien.txt"
 
# Enregistrement de la transcription
with open(entretien_transcrit , 'w', encoding='utf-8') as f:
    for segment in result["segments"]:
        start_time = convertir(segment['start'])
        end_time = convertir(segment['end'])
        f.write(f"{start_time} - {end_time}: {segment['text']}\n")

In [27]:
print(open("entretien.txt","r").read())

00:00:00 - 00:00:03:  On les parant qui était là, elle n'est pas qu'on prie cette réalisation.
00:00:03 - 00:00:05:  Et très vite, ce sont orientés.
00:00:05 - 00:00:07:  Une fois que la maison était en cours, en fait,
00:00:07 - 00:00:09:  sur tous les problèmes de qualité de vie.
00:00:09 - 00:00:13:  Dans lesquels il y avait le social,
00:00:13 - 00:00:17:  la formation, la vie à l'hôpital.
00:00:17 - 00:00:18:  La prise en charge, etc.
00:00:18 - 00:00:22:  Pas toujours, donc je vous demande de répondre de sa part.
00:00:22 - 00:00:24:  Pas toujours avec la corps, d'ailleurs, des médecins.
00:00:24 - 00:00:28:  C'est-à-dire que on n'était pas en conflit,
00:00:28 - 00:00:29:  mais non non, c'est pas ça du tout.
00:00:29 - 00:00:31:  On n'était pas toujours.
00:00:31 - 00:00:32:  On connaît ça des plus...
00:00:32 - 00:00:34:  C'était en 1990.
00:00:34 - 00:00:36:  Et à ce moment-là, la vie d'atrice à Gustavouille-Cier,
00:00:36 - 00:00:37:  elle avait une place importante.
00:00:37

Tester l'erreur 

- https://github.com/jitsi/jiwer